In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import datetime
import sys
from tqdm import tqdm
import os
import random
import pickle
from glob import glob
import gc
from multiprocessing import Pool, cpu_count

import warnings
warnings.filterwarnings('ignore')

sys.path.append('/kaggle')
from utils.xgb import fit_xgb, inference_xgb
from utils.metric import compute_comptetition_metric
from utils.feature_contena import Features
from utils.pandas_utils import reduce_mem_usage

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
config = f"""
execution:
    exp_id: exp_015
    debug: False
    submit: False
    multiprocessing: True

dataset: 
    competition_dir: /kaggle/input/child-mind-institute-detect-sleep-states
    cv_split_path: /kaggle/input/cv_split/train_folds.csv
    train_base_path: /kaggle/input/train_base/train_base.csv
    step_csv_dir: /kaggle/input/save_series_csv/csvs

feature:
    agg_freq: 24 # [step]

xgboost:    
    objective: "binary:logistic"
    learning_rate: 0.1  # 0.01で固定。学習時間とのトレードオフ
    reg_alpha: 0.02  # L1正則化。0.1が推奨。
    reg_lambda: 0.2  # L2正則化。0.1が推奨
    random_state: 42
    max_depth: 5  # 3-8。7くらいでいい。
    colsample_bytree: 0.7  # カラムが多い時は少なめ(0.4とか)にする。
    
seed: 46
"""

CFG = yaml.load(config, Loader=yaml.SafeLoader)

CFG["output_dir"] = f"/kaggle/output/{CFG['execution']['exp_id']}"
os.makedirs(CFG["output_dir"], exist_ok=True)

In [3]:
train = pd.read_csv(os.path.join("/kaggle/output", os.path.join(CFG["execution"]["exp_id"], "oof.csv")))
train.head()

,target,step,series_id,fold,oof_stage1,STAGE_stage1,raw_oof,oof_stage2,STAGE_stage2,oof
0,1,11.5,038441c925bb,2,NaN,2,0.999819,NaN,NaN,NaN
1,1,35.5,038441c925bb,2,NaN,2,0.999644,NaN,NaN,NaN
2,1,59.5,038441c925bb,2,NaN,2,0.993865,NaN,NaN,NaN
3,1,83.5,038441c925bb,2,NaN,2,0.995005,NaN,NaN,NaN
4,1,107.5,038441c925bb,2,NaN,2,0.994551,NaN,NaN,NaN


In [ ]:
dfs = []
for series_id in tqdm(train["series_id"].unique()):
    df = pd.read_csv(f"{CFG['dataset']['step_csv_dir']}/{series_id}.csv")

    DAY_STEPS = 60 * 60 * 24 // 5
    n_days = int(len(df) // DAY_STEPS) + 1
    df["same_count"] = 0
    for day in range(-n_days, n_days + 1):
        if day == 0:
            continue
        df["anglez_diff"] = df["anglez"].diff(DAY_STEPS * day)
        df["anglez_diff"] = df["anglez_diff"].fillna(1)
        df["same_count"] += (df["anglez_diff"] == 0).astype(int)
            
    df["group"] = df["target"].diff().abs().cumsum()
    start_steps = df.groupby("group").head(1)["step"].values
    end_steps = df.groupby("group").tail(1)["step"].values

    for start_step, end_step in zip(start_steps, end_steps):
        MERGIN = 12 * 120 # 120min
        group_df = df[(df["step"] >= start_step - MERGIN) & (df["step"] <= end_step + MERGIN)]

        if group_df["target"].values[len(group_df) // 2] == 1:
            continue

        fig, axs = plt.subplots(3, 1, figsize=(20, 7))
        axs[0].plot(group_df["step"], group_df["enmo"])
        axs[0].axvline(start_step, color="red")
        axs[0].axvline(end_step, color="red")
        axs[0].set_ylabel("enmo")
        axs[1].plot(group_df["step"], group_df["anglez"])
        axs[1].axvline(start_step, color="red")
        axs[1].axvline(end_step, color="red")
        axs[1].set_ylabel("anglez")
        axs[2].plot(group_df["step"], group_df["same_count"])
        axs[2].axvline(start_step, color="red")
        axs[2].axvline(end_step, color="red")
        axs[2].set_ylabel("same_count")
        plt.suptitle(f"series_id: {series_id}")
        plt.tight_layout()

        savepath = f"/kaggle/notebook/gt_images/{series_id}/{start_step}-{end_step}.png"
        os.makedirs(os.path.dirname(savepath), exist_ok=True)
        plt.savefig(savepath)
        plt.close()